# Notebook Overview: Exploring LLM Agents and Tools with Amazon Bedrock

This notebook is a practical exploration of **Large Language Model (LLM) agents** and their integration with memory, tools, and reasoning frameworks using **Amazon Bedrock**. The activities are designed to demonstrate advanced techniques for augmenting LLMs via LangChain, a framework for building applications powered by language models.

### Objectives:
1. **Introduction to Chain of Thought (CoT) and Tree of Thoughts (ToT):**
   - Learn how multi-step reasoning enhances LLM performance.
   - Implement sequential reasoning using CoT techniques.

2. **Memory Integration:**
   - Understand different types of memory: sensory, short-term, and long-term.
   - Implement long-term memory using vector databases.
   - Use short-term memory to maintain conversational context.

3. **Tool Integration:**
   - Explore how external tools (e.g., search APIs, calculators) can extend LLM capabilities.
   - Build and use custom tools to answer complex queries.

4. **Agent Construction:**
   - Define agents that combine reasoning, tools, and memory for advanced workflows.
   - Implement ReAct (Reasoning and Acting) agents using Amazon Bedrock.

5. **Structuring Outputs:**
   - Learn to format outputs using structured data models (e.g., Pydantic).

In [ ]:
import os
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_aws import BedrockLLM
from langchain.prompts import PromptTemplate
from langchain_aws import ChatBedrock
import ipywidgets as widgets
import boto3
import json
import sys

from dotenv import load_dotenv


def load_config(config_path):
    with open(config_path, "r") as f:
        return json.load(f)


config = load_config("chain_config.json")
load_dotenv(".env")
aws_region = "us-east-1"

bedrock_runtime_client = boto3.client("bedrock-runtime", region_name=aws_region)
bedrock_management_client = boto3.client('bedrock', region_name=aws_region)
bedrock_agent_client = boto3.client('bedrock-agent', region_name=aws_region)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=aws_region)
cloudformation_client = boto3.client('cloudformation', region_name=aws_region)

boto3.__version__

In [ ]:
%load_ext autoreload
%autoreload 2

# Planning

![images/Capture-2024-11-23-235604.png](<images/Capture-2024-11-23-235604.png>)

You may have encountered different strategies designed to enhance the performance of large language models, ranging from giving them guidance to lightheartedly "motivating" them. A widely recognized method is the "chain of thought" technique, where the model is prompted to reason through problems step by step, allowing it to identify and fix errors as it goes. This strategy has been further developed into more sophisticated approaches, such as "chain of thought with self-consistency," and extended into the broader "tree of thoughts" framework. In this generalized approach, multiple lines of reasoning are generated, re-examined, and integrated to arrive at a more robust and accurate response.

In [ ]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Amazon",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', []) if "Nova" not in model['modelName'] 
    ],
    value='amazon.titan-text-express-v1',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

### Tree of Thoughts

The [@astropomeai tutorial](https://medium.com/@astropomeai/implementing-the-tree-of-thoughts-in-langchains-chain-f2ebc5864fac) on Tree of Thoughts is used as basis of this exercise but expanded with LLMOps tools.

In [ ]:
cot_step1_prompt = PromptTemplate(template=config["step1"]["prompt"], input_variables=["input", "perfect_factors"])
cot_step2_prompt = PromptTemplate(template=config["step2"]["prompt"], input_variables=["solutions"])
cot_step3_prompt = PromptTemplate(template=config["step3"]["prompt"], input_variables=["review"])
cot_step4_prompt = PromptTemplate(template=config["step4"]["prompt"], input_variables=["deepen_thought_process"])

In [ ]:
model = agent_foundation_model_selector.value

chain1 = LLMChain(
    llm=ChatBedrock(temperature=0, model=model),
    prompt=cot_step1_prompt,
    output_key="solutions"
)

chain2 = LLMChain(
    llm=ChatBedrock(temperature=0, model=model),
    prompt=cot_step2_prompt,
    output_key="review"
)

chain3 = LLMChain(
    llm=ChatBedrock(temperature=0, model=model),
    prompt=cot_step3_prompt,
    output_key="deepen_thought_process"
)

chain4 = LLMChain(
    llm=ChatBedrock(temperature=0, model=model),
    prompt=cot_step4_prompt,
    output_key="ranked_solutions"
)

In [ ]:
overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

params = {
    "input": "human colonization of Mars",
    "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
}

response = overall_chain.invoke(params)
print(response['ranked_solutions'])

Your task is now to rewrite the above logic by using the `|` operator. You can also take advantage of `StrOutputParser`, and you can import it in this way: `from langchain_core.output_parsers import StrOutputParser`.

<details>
<summary>Click here for the solution</summary>
    
```python
from langchain_core.output_parsers import StrOutputParser

llm = ChatBedrock(temperature=0, model=model)

chain1 = cot_step1_prompt | llm | {"solutions": StrOutputParser()}
chain2 = cot_step2_prompt | llm | {"review": StrOutputParser()}
chain3 = cot_step3_prompt | llm | {"deepen_thought_process": StrOutputParser()}
chain4 = cot_step4_prompt | llm | {"ranked_solutions": StrOutputParser()}

overall_chain = chain1 | chain2 | chain3 | chain4

params = {
    "input": "human colonization of Mars",
    "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
}

result = overall_chain.invoke(params)
print(result['ranked_solutions'])
```
    
</details>

### ReAct prompt overview

Let's review [ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react) prompt as it's defined in Langchain.

In [ ]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Anthropic",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', [])
    ],
    value='anthropic.claude-3-5-sonnet-20240620-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

In [ ]:
react_agent_prompt = PromptTemplate(
    template=config["react_agent"]["prompt"], 
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

print(react_agent_prompt.template)

In [ ]:
from langchain_utils import get_trivia_react_agent

llm = ChatBedrock(
    temperature=0, 
    model=agent_foundation_model_selector.value
)

agent = get_trivia_react_agent(llm, react_agent_prompt)

In [ ]:
agent.invoke({"input": "What is the capital of France?"})

In [ ]:
agent.invoke({"input": "What is the capital of Oswanda?"})

In [ ]:
agent.invoke({"input": "What is the capital of Althera?"})

### Question: Why does the Agent refuse to fully trust the tool in certain cases?

### Self-ask with search

Let's review [self-ask with search](https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search) as it's defined in Langchain.

In [ ]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Anthropic",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', []) if "Nova" not in model['modelName']
    ],
    value='anthropic.claude-3-haiku-20240307-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

In [ ]:
self_ask_with_search_prompt = PromptTemplate(
    template=config["self_ask_with_search"]["prompt"], 
    input_variables=["input", "agent_scratchpad"]
)

print(self_ask_with_search_prompt.template)

Approaches to read next:

**Reflexion** ([Shinn & Labash 2023](https://arxiv.org/abs/2303.11366)) is a framework to equips agents with dynamic memory and self-reflection capabilities to improve reasoning skills.

**Chain of Hindsight** (CoH; [Liu et al. 2023](https://arxiv.org/abs/2302.02676)) encourages the model to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback.

In [ ]:
from langchain.agents import SelfAskWithSearchChain
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain.tools import tool
import logging

model = agent_foundation_model_selector.value

@tool("Intermediate Answer")
def simulated_search(query: str) -> str:
    """
    This tool searching for information on internet.
    """
    return "I'm sorry, I couldn't find information on that topic."


simulated_search

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.05,
    check_every_n_seconds=0.5,
    max_bucket_size=5,
)

model = agent_foundation_model_selector.value

llm = ChatBedrock(
    temperature=0,
    model=model,
    rate_limiter=rate_limiter
)

In [ ]:
agent = create_self_ask_with_search_agent(
    llm=llm,
    tools=[simulated_search],
    prompt=self_ask_with_search_prompt
)

self_ask_with_search_executor = AgentExecutor(
    agent=agent,
    tools=[simulated_search],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

question = "How far away is the colonization of Mars?"

answer = self_ask_with_search_executor.invoke({"input": question})
print(answer)

# Memory

![./images/memory.png](<./images/memory.png>)


- **Sensory Memory:** This component of memory captures immediate sensory inputs, like what we see, hear, or feel. In the context of prompt engineering and AI models, a prompt serves as a transient input, similar to a momentary touch or sensation. It's the initial stimulus that triggers the model's processing.

- **Short-Term Memory:** Short-term memory holds information temporarily, typically related to the ongoing task or conversation. In prompt engineering, this equates to retaining the recent chat history. This memory enables the agent to maintain context and coherence throughout the interaction, ensuring that responses align with the current dialogue.

- **Long-Term Memory:** Long-term memory stores both factual knowledge and procedural instructions. In AI models, this is represented by the data used for training and fine-tuning. Additionally, long-term memory supports the operation of RAG frameworks, allowing agents to access and integrate learned information into their responses. It's like the comprehensive knowledge repository that agents draw upon to generate informed and relevant outputs.

### Adding long-term memory

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.vectorstores import DuckDB 

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byOutputModality="EMBEDDING",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', [])
    ],
    value='amazon.titan-embed-g1-text-02',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

In [ ]:
import duckdb 

connection = duckdb.connect("data.db")

bedrock_embeddings = BedrockEmbeddings(
    model_id=agent_foundation_model_selector.value, 
    region_name=aws_region
)

**Instructions:**

1. **Use `WebBaseLoader` to Load a Document:**
   - Fetch the content of the webpage at `https://aws.amazon.com/ai/our-story` using `WebBaseLoader`.

2. **Set up `RecursiveCharacterTextSplitter`:**
   - Create an instance of `RecursiveCharacterTextSplitter` with the following parameters:
     - `chunk_size=1000` (each chunk should have a maximum size of 1000 characters).
     - `chunk_overlap=200` (each chunk should overlap the previous one by 200 characters).

3. **Call `split_documents`:**
   - Use the `split_documents` method from `RecursiveCharacterTextSplitter` to process the document loaded with `WebBaseLoader`.
   
<details>
<summary>Click here for the solution</summary>
    
```python
url = "https://aws.amazon.com/ai/our-story"
loader = WebBaseLoader(url)
docs = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)
```
    
</details>

**Instructions:**

1. **Create a Vector Store Using `from_documents`:**
   - Use the `DuckDB.from_documents` method to initialize a vector store.
   - Pass the following parameters:
     - `documents`: A variable containing the preprocessed documents.
     - `embedding`: A variable containing the embedding model (e.g., `bedrock_embeddings`).

2. **Set Up a Retriever Using `as_retriever`:**
   - Call the `as_retriever` method on the created vector store to transform it into a retriever.
   
<details>
<summary>Click here for the solution</summary>
    
```python
duckdb_vectorstore = DuckDB.from_documents(
    connection=connection,
    documents=documents,
    embedding=bedrock_embeddings
)
retriever = duckdb_vectorstore.as_retriever()
```
    
</details>

### Example: Using OpenSearch as a Vector Store in LangChain

LangChain supports various databases as vector stores, including OpenSearch. Below is an example of setting up OpenSearch as a vector store for use in LangChain.

<details>
<summary>Click here for the Code Example:</summary>
    
```python
from opensearchpy import AWSV4SignerAuth
from opensearchpy.connection import RequestsHttpConnection
import boto3

HOST = "https://<your-opensearch-endpoint>.aoss.amazonaws.com"
INDEX_NAME = "aws-ai-our-story"
INDEX_DIMENSION = 1024
MAX_TOKENS = 512

credentials = boto3.Session().get_credentials()
awsauth = AWSV4SignerAuth(credentials, aws_region, "aoss")

opensearch_vectorstore = OpenSearchVectorSearch.from_documents(
    documents=documents,
    embedding=bedrock_embeddings,
    opensearch_url=HOST,
    index_name=INDEX_NAME,
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

index_mapping = {
    "settings": {"index": {"knn": True, "knn.algo_param.ef_search": 512}},
    "mappings": {
        "properties": {
            "vector_field": {
                "type": "knn_vector",
                "dimension": 128,
                "method": {
                    "name": "hnsw",
                    "space_type": "l2",
                    "engine": "nmslib",
                    "parameters": {"ef_construction": 512, "m": 16},
                },
            }
        }
    },
}

if not opensearch_vectorstore.client.indices.exists(index=INDEX_NAME):
    opensearch_vectorstore.client.indices.create(index=INDEX_NAME, body=index_mapping)

retriever = opensearch_vectorstore.as_retriever()
```
    
</details>

In [ ]:
query = "What does AWS offer for Responsible AI?"
results = retriever.get_relevant_documents(query)

for result in results:
    print(result.page_content)

### Adding short-term memory

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Anthropic",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', []) if "Nova" not in model['modelName']
    ],
    value='anthropic.claude-3-haiku-20240307-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

In [ ]:
system_message = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=[],
        template="You are a helpful assistant"
    )
)

chat_history_placeholder = MessagesPlaceholder(
    variable_name="chat_history",
    optional=True
)

human_message = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["input"],
        template="{input}"
    )
)

messages = [system_message, chat_history_placeholder, human_message]
prompt = ChatPromptTemplate.from_messages(messages)
prompt.messages

**Instructions:**

1. **Model Selection:**
   - Retrieve the selected model from a UI component or variable, such as `agent_foundation_model_selector`.
   - Assign the selected model to a variable for further use.

2. **Initialize the Model:**
   - Use the selected model to create an instance of a chat-based language model (e.g., `ChatBedrock`).
   - Set appropriate parameters, such as `temperature`, to control the model's behavior.

3. **Create a Runnable Pipeline:**
   - Combine the `prompt` with the initialized model to create a runnable pipeline.
   - Use the pipeline to process input through the prompt and model.
   

<details>
<summary>Click here for the solution</summary>
    
```python
model = agent_foundation_model_selector.value
model = ChatBedrock(temperature=0, model=model)

runnable = prompt | model
```
    
</details>

In [ ]:
model = agent_foundation_model_selector.value
model = ChatBedrock(temperature=0, model=model)

runnable = prompt | model

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


agent_with_chat_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
session_id = "<foo>"

In [ ]:
response = agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": session_id}},
)
response.content

In [ ]:
response = agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": session_id}},
)
response.content

Print the Content of a Session and Experiment with Other Session IDs

<details>
<summary>Click here for the solution</summary>
    
```python
print(str(get_session_history(session_id)))
```
    
</details>

# Tools in AI Systems

![./images/tools.png](<./images/tools.png>)

The **earliest** implementations of tool-augmented AI architectures were introduced in 2022 by A21Labs.

These tools can be categorized as either **neural**, such as deep learning models, or **symbolic**, like a mathematical calculator, currency converter, or weather API.

Examples of such architectures in action include ChatGPT’s [Plugins](https://openai.com/blog/chatgpt-plugins) and the **OpenAI API** [function calling](https://platform.openai.com/docs/guides/gpt/function-calling), both of which enable LLMs to effectively leverage external tools.

### Memory as a retrieval tool

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="aws_ai_story_search",
    description=(
        "Search for information about AWS AI and its initiatives. "
        "Use this tool to answer questions related to AWS AI story and related content."
    ),
)

### LangChain Tools

Discover more tools at [LangChain Integrations](https://python.langchain.com/docs/integrations/tools).

LangChain provides a variety of agent types, each tailored to different scenarios. Below are some of the key agents offered:

- **Zero-shot ReAct:** 
  - Operates using the ReAct framework, selecting tools based only on their descriptions.
  - Requires clear descriptions for each tool and is highly adaptable to diverse tasks.

- **Structured Input ReAct:** 
  - Designed for tools with multiple input parameters.
  - Ideal for complex operations like web browsing, as it leverages the tools' argument schemas for structured inputs.

- **OpenAI Functions:** 
  - Tailored for models optimized for function calling, such as `gpt-3.5-turbo-0613` and `gpt-4-0613`.
  - Used in creating our first agent example earlier.

- **Conversational Agent:** 
  - Focused on dialogue-based applications, using the ReAct framework for tool selection.
  - Incorporates memory to retain context from previous interactions.

- **Self-ask with Search:** 
  - Uses a single tool, "Intermediate Answer," to retrieve factual information.
  - Based on the original self-ask with search methodology.

- **ReAct Document Store:** 
  - Interacts with a document store through the ReAct framework.
  - Requires tools like "Search" and "Lookup," closely resembling the approach used in the original ReAct paper's Wikipedia example.

For further details on these agents, check out [this blog post](https://nanonets.com/blog/langchain/).

In [ ]:
from langchain.agents import initialize_agent, AgentType
from langchain_aws import ChatBedrockConverse

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Amazon",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', []) if "Nova" in model['modelName']
    ],
    value='amazon.nova-lite-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

In [ ]:
llm = ChatBedrockConverse(
    temperature=0.0,
    model=agent_foundation_model_selector.value
)

**Instructions:**

1. **Provide the Necessary Inputs:**
   - Use a `retriever_tool` as the tool the agent will use for retrieving information.
   - Pass an `llm` (Language Learning Model) as the core reasoning model.

2. **Set Up the Agent Type:**
   - Use `AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` to configure the agent for structured chat interactions.

3. **Enable Verbose Output:**
   - Set the `verbose` parameter to `True` to enable detailed logging of the agent's activities.

4. **Write the Code:**
   - Implement the code to create the agent using the `initialize_agent` function.

<details>
<summary>Click here for the solution</summary>

```python
agent_chain = initialize_agent(
    [retriever_tool],
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
```

</details>

In [ ]:
initialize_agent?

Test the agent by invoking it with different Nova language models.

In [ ]:
agent_chain.invoke(
    {"input": "What services does AWS AI offer?"},
)

In [ ]:
agent_chain.invoke(
    "How has AWS AI influenced global AI research trends, and what are the long-term implications for industries like healthcare and finance?"
)

### Custom tools and reasoning

Here is a full guide on how to create [custom tools for LangChain](https://python.langchain.com/docs/modules/agents/tools/custom_tools)

In [ ]:
from langchain.tools import BaseTool, StructuredTool, tool

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Anthropic",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', [])
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)
agent_foundation_model_selector

### Tool Description: Brave Search Integration

The `BraveSearch` tool integrates Brave's search engine capabilities into your application, allowing you to perform web searches programmatically.

1. **Initialization:**
   - The tool is initialized using an API key (`api_key`).
   - Additional search parameters, such as `count`, can be passed through `search_kwargs`.

2. **Usage:**
   - The `count` parameter in `search_kwargs` specifies the number of search results to return (in this case, 3).

3. **Functionality:**
   - Performs web searches using the Brave search engine.
   - Provides a convenient way to retrieve relevant results for various queries.

This tool is ideal for integrating web search functionality into applications that require programmatic access to search results.

In [ ]:
from langchain_community.tools import BraveSearch

api_key= "BSAj2ZNvwVxXtx1HzY05vfDVSNizcgi"
brave_tool = BraveSearch.from_api_key(api_key=api_key, search_kwargs={"count": 3})

In [ ]:
brave_tool.description, brave_tool.args,

In [ ]:
brave_tool.run(tool_input="AWS AI services")

### Tool Description: Scrape and Convert to Markdown

The `scrape_and_convert_to_markdown_tool` is a custom tool that performs the following tasks:

1. **Input:** Accepts a URL as a string.
2. **Sanitization:** Ensures the URL has a valid scheme (e.g., `https://`). If not, it prepends `https://` to the input.
3. **Web Scraping:** Fetches the webpage content using the `requests` library.
4. **HTML Parsing:** Extracts the body of the webpage using `BeautifulSoup`.
5. **Markdown Conversion:** Converts the HTML content to Markdown format using the `markdownify` library with ATX-style headings.
6. **Output:** Returns the converted Markdown content or an error message if the scraping process fails.

This tool is useful for retrieving and converting webpage content into a Markdown-readable format for further use in documentation, content management, or analysis.

In [ ]:
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify
from langchain.tools import tool
from urllib.parse import urlparse, urlunparse

@tool
def scrape_and_convert_to_markdown_tool(url: str) -> str:
    """
    Scrapes the content of the given URL and converts it to Markdown format.
    """
    try:
        parsed_url = urlparse(url.strip())
        if not parsed_url.scheme:
            sanitized_url = f"https://{url.strip()}"
        else:
            sanitized_url = urlunparse(parsed_url)

        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        response = requests.get(sanitized_url, headers=headers)
        response = requests.get(sanitized_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.body
        markdown_content = markdownify(str(content), heading_style="ATX")

        return markdown_content
    except requests.exceptions.RequestException as e:
        return f"Error fetching the URL: {e}"
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:
scrape_and_convert_to_markdown_tool.description, scrape_and_convert_to_markdown_tool.args

In [ ]:
scrape_and_convert_to_markdown_tool.run(tool_input="https://aws.amazon.com/ai/services/")[:2000]

**Instructions:**

1. Write a Python function decorated with `@tool`.
2. The tool should take a string as input.
3. The tool should return the length of the input string.

<details>
<summary>Click here for the solution</summary>

```python
from langchain.tools import tool

@tool
def calculate_string_length(input_string: str) -> int:
    """
    Calculates the length of the input string.
    """
    return len(input_string)
```

</details>

In [ ]:
calculate_string_length.description, calculate_string_length.args,

In [ ]:
calculate_string_length.run(tool_input="-"*10)

**Instructions:**

1. Write a Python function and decorate it with `@tool`.
2. The tool should accept a string as input.
3. The tool should return the number of uppercase letters in the input string.

<details>
<summary>Click here for the solution</summary>

```python
from langchain.tools import tool

@tool
def calculate_uppercase_tool(input_string: str) -> int:
    """
    Calculates the number of uppercase letters in the input string.
    """
    return sum(1 for char in input_string if char.isupper())
```

</details> 


In [ ]:
calculate_uppercase_tool.description, calculate_uppercase_tool.args,

In [ ]:
calculate_uppercase_tool.run(tool_input="A"*4+"b"*5)

In [ ]:
llm = ChatBedrock(
    temperature=0.0,
    model=agent_foundation_model_selector.value
)

**Instructions:**

1. **Provide the Necessary Inputs:**
   - Create a `tools` list to define the actions the agent can perform.
   - Use an `llm` (Language Model) as the core reasoning component.

2. **Set Up the Agent Type:**
   - Configure the agent with `AgentType.ZERO_SHOT_REACT_DESCRIPTION` for zero-shot reasoning and structured responses.

3. **Enable Verbose Output:**
   - Set the `verbose` parameter to `True` to enable detailed logging and provide insights into the agent's actions.

4. **Write the Code:**
   - Use the `initialize_agent` function to set up the agent with the specified tools, language model, and configuration.

<details>
<summary>Click here for the solution</summary>

```python
from langchain.agents import initialize_agent, AgentType

tools = [brave_tool, calculate_string_length, calculate_uppercase_tool, scrape_and_convert_to_markdown_tool]
agent_chain = initialize_agent(
    llm=llm,
    tools=tools,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)
```

</details>

In [ ]:
for tool in tools:
    print("Tool Name:", tool.name)
    print("Tool Description:", tool.description)
    print()

In [ ]:
query = '\n'.join([
    "Find the description of AWS AI services.",
    "Scrape the content of the relevant URL(s).",
    "Convert the scraped content to Markdown format.",
    "Calculate the length of the Markdown description.",
])

response = agent_chain.invoke(
    {"input": query}
)

print(response)

### Recap of Activities: Building a LangChain-Powered System with Amazon Bedrock

This recap provides an overview of the tools, memory systems, and configurations we implemented to create a powerful system leveraging **LangChain** and **Amazon Bedrock**. This system integrates reasoning, information retrieval, and text processing to deliver efficient and context-aware interactions.

---

### Tools

#### 1. **Brave Search Tool**
We integrated the **Brave Search API** to programmatically retrieve web content. The following code demonstrates how we set up the Brave Search tool:

```python
from langchain_community.tools import BraveSearch

api_key = "<your_api_key>"
brave_tool = BraveSearch.from_api_key(api_key=api_key, search_kwargs={"count": 3})
```

#### 2. **Custom Tools**
We created several tools to enhance the system's functionality:

- **Calculate String Length**:
  A tool to compute the length of an input string:
  ```python
  from langchain.tools import tool

  @tool
  def calculate_length_tool(input_string: str) -> int:
      """Calculates the length of the input string."""
      return len(input_string)
  ```

- **Count Uppercase Letters**:
  A tool to count the number of uppercase letters in a string:
  ```python
  @tool
  def calculate_uppercase_tool(input_string: str) -> int:
      """Counts the number of uppercase characters in the input string."""
      return sum(1 for char in input_string if char.isupper())
  ```

- **Scrape and Convert to Markdown**:
  A tool to scrape content from a webpage and convert it to Markdown format:
  ```python
  import requests
  from bs4 import BeautifulSoup
  from markdownify import markdownify
  from langchain.tools import tool
  from urllib.parse import urlparse, urlunparse

  @tool
  def scrape_and_convert_to_markdown_tool(url: str) -> str:
      """
      Scrapes the content of the given URL and converts it to Markdown format.
      """
      try:
          parsed_url = urlparse(url.strip())
          if not parsed_url.scheme:
              sanitized_url = f"https://{url.strip()}"
          else:
              sanitized_url = urlunparse(parsed_url)

          headers = {"User-Agent": "Mozilla/5.0"}
          response = requests.get(sanitized_url, headers=headers)
          response.raise_for_status()
          soup = BeautifulSoup(response.text, 'html.parser')
          content = soup.body
          if content:
              markdown_content = markdownify(str(content), heading_style="ATX")
              return markdown_content
          else:
              return "No content found in the body of the page."
      except requests.exceptions.RequestException as e:
          return f"Error fetching the URL: {e}"
      except Exception as e:
          return f"An error occurred: {e}"
  ```

---

### Memory Systems

#### 1. **Long-Term Memory**
We implemented long-term memory using **DuckDB** as a vector database to store and retrieve document embeddings. This allows the system to retain and query relevant knowledge efficiently.

```python
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import DuckDB

# Split content into chunks
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
).split_documents(docs)

# Create DuckDB vector store
duckdb_vectorstore = DuckDB.from_documents(
    documents=documents,
    embedding=bedrock_embeddings,
    connection=connection
)

# Initialize retriever
retriever = duckdb_vectorstore.as_retriever()

# Create retriever tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    name="aws_ai_story_search",
    description="Searches for information about AWS AI services and initiatives."
)
```

#### 2. **Short-Term Memory**
To manage short-term memory, we used **ChatMessageHistory** to store and retrieve conversation context during interactions:

```python
from langchain_community.chat_message_histories import ChatMessageHistory

message_history = ChatMessageHistory()
```

#### 3. **Sensory Memory**
We used **Amazon Bedrock** for real-time prompt-based interaction, leveraging predefined templates for context-aware responses:

```python
from langchain.prompts import PromptTemplate
from langchain_aws import ChatBedrock
from langchain import hub

# Load a predefined prompt template
prompt = hub.pull("hwchase17/openai-functions-agent")

# Initialize the Bedrock language model
model = ChatBedrock(temperature=0.7)
runnable = prompt | model
```

# Agent

### All tools together

![images/agent.png](<images/agent.png>)

### Defining an agent with tools and memory

In [ ]:
agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Anthropic",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', [])
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)

agent_foundation_model_selector

In [ ]:
model = agent_foundation_model_selector.value
llm = ChatBedrock(model=model, temperature=0)

In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.tools.retriever import create_retriever_tool
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import DuckDB
import duckdb


con = duckdb.connect("data.db")
bedrock_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0", 
    region_name=aws_region
)

def get_retriever(con, embed, url, chunk_size=500, chunk_overlap=100):
    loader = WebBaseLoader(url)

    documents = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    ).split_documents(loader.load())
    
    duckdb_vectorstore = DuckDB.from_documents(
        connection=con,
        documents=documents,
        embedding=embed
    )

    return duckdb_vectorstore.as_retriever()


retriever_tool = create_retriever_tool(
    retriever=get_retriever(
        con=con, 
        embed=bedrock_embeddings,
        url="https://aws.amazon.com/ai/our-story"
    ),
    name="aws_ai_story_search",
    description=(
        "Search for information about AWS AI and its initiatives. "
        "Use this tool to answer questions related to AWS AI story and related content."
    ),
)

In [ ]:
from langchain_community.tools import BraveSearch

api_key = "BSAj2ZNvwVxXtx1HzY05vfDVSNizcgi"

brave_tool = BraveSearch.from_api_key(
    api_key=api_key, 
    search_kwargs={"count": 3}
)

In [ ]:
from langchain.tools import tool

@tool
def scrape_and_convert_to_markdown_tool(url: str) -> str:
    """
    Scrapes the content of the given URL and converts it to Markdown format.
    """
    try:
        parsed_url = urlparse(url.strip())
        if not parsed_url.scheme:
            sanitized_url = f"https://{url.strip()}"
        else:
            sanitized_url = urlunparse(parsed_url)

        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
        response = requests.get(sanitized_url, headers=headers)
        response = requests.get(sanitized_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        content = soup.body
        markdown_content = markdownify(str(content), heading_style="ATX")

        return markdown_content
    except requests.exceptions.RequestException as e:
        return f"Error fetching the URL: {e}"
    except Exception as e:
        return f"An error occurred: {e}"


@tool
def calculate_uppercase_tool(input_text: str) -> int:
    """
    Counts the number of uppercase letters in the provided input string.

    Args:
        input_text: The string to analyze.

    Returns:
        The count of uppercase characters in the input string.
    """
    return sum(1 for c in input_text if c.isupper())


@tool
def calculate_length_tool(input_text: str) -> int:
    """
    Calculates the total number of characters in the provided input string.

    Args:
        input_text: The string to analyze.

    Returns:
        The length of the input string, including spaces and special characters.
    """
    return len(input_text)

In [ ]:
tools = [
    retriever_tool, 
    brave_tool, 
    scrape_and_convert_to_markdown_tool, 
    calculate_length_tool, 
    calculate_uppercase_tool
]

for tool in tools:
    des = tool.description.split("\n")
    print(f"{tool.name}: {des[0]}")

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
store

In [ ]:
from langchain.prompts import PromptTemplate

def load_config(config_path):
    with open(config_path, "r") as f:
        return json.load(f)

config = load_config("chain_config.json")

react_agent_with_chat_history_prompt = PromptTemplate(
    template=config["react_agent_with_chat_history"]["prompt"], 
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

print(react_agent_with_chat_history_prompt.template)

### Instructions

1. **Create a React Agent:**
   - Use the `create_react_agent` function to initialize a React agent.
   - Pass the `llm` (language model), `tools`, and `react_agent_with_chat_history_prompt` as parameters.

2. **Set Up the Agent Executor:**
   - Create an `AgentExecutor` to manage the agent's execution.
   - Configure the `AgentExecutor` with:
     - The React agent (`agent`).
     - A list of tools (`tools`).
     - `verbose=True` for detailed logging.
     - `handle_parsing_errors=True` to gracefully handle any parsing issues during execution.

3. **Enable Chat History:**
   - Wrap the `AgentExecutor` in a `RunnableWithMessageHistory` to include session history in the interaction.
   - Configure the keys for managing input, history, and output:
     - `input_messages_key="input"`: Key for the input messages.
     - `history_messages_key="chat_history"`: Key for storing session history.
     - `output_messages_key="output"`: Key for the agent's output messages.

---

### Example Solution:

<details>
<summary>Click here for the solution</summary>

```python
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor
from langchain.agents import create_react_agent

# Step 1: Create a React Agent
agent = create_react_agent(llm, tools, react_agent_with_chat_history_prompt)

# Step 2: Set up the Agent Executor
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True
)

# Step 3: Enable Chat History
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="output"
)
```

</details>

In [ ]:
session_id = "123"

### Running an agent

In [ ]:
instruction = "What is the current population of Italy?"
response = agent_with_chat_history.invoke(
    {"input": f"Hi! I'm bob. {instruction}"},
    config={"configurable": {"session_id": session_id}},
    verbose=True
)

print(response['output'])

In [ ]:
response = agent_with_chat_history.invoke(
    {"input": "What's my name?"},
    config={"configurable": {"session_id": session_id}},
)

print(response['output'])

In [ ]:
instruction = "In all the next questions I will ask you, you need to call me with my name before answering, is it ok for you?"
response = agent_with_chat_history.invoke(
    {"input": instruction},
    config={"configurable": {"session_id": session_id}},
)

print(response['output'])

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": "How many under 18 in Italy?"
    },
    config={"configurable": {"session_id": session_id}},
)

print(response['output'])

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": "What are the AI services provided by AWS? Does AWS have a region in Italy?"
    }, 
    config={"configurable": {"session_id": session_id}},
)

print(response['output'])

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": "What is the weather today in Italy?"
    }, 
    config={"configurable": {"session_id": session_id}},
)

print(response['output'])

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": "Make a quick summary of what we have discussed today"
    }, 
    config={"configurable": {"session_id": session_id}},
)

print(response['output'])

In [ ]:
response = agent_with_chat_history.invoke(
    {
        "input": "Calculate the length of this summary and the uppercase characters of this summary"
    }, 
    config={"configurable": {"session_id": session_id}},
)

print(response['output'])

### Structuring Outputs

Sometimes, the output of a language model needs to serve as input for another program. In such cases, it's important to follow a specific structure, such as creating a JSON object that the target program can interpret. In this section, we will walk through an example of how to structure the output of a language model properly.

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain_aws import ChatBedrock
from pydantic import BaseModel, Field
import ipywidgets as widgets

agent_foundation_model_selector = widgets.Dropdown(
    options=[
        (model['modelName'], model['modelId']) 
        for model in bedrock_management_client.list_foundation_models(
            byProvider="Anthropic",
            byOutputModality="TEXT",
            byInferenceType="ON_DEMAND"
        ).get('modelSummaries', [])
    ],
    value='anthropic.claude-3-sonnet-20240229-v1:0',
    description='FM:',
    disabled=False,
)

agent_foundation_model_selector

In [ ]:
class DesiredStructure(BaseModel):
    question: str = Field(description="The question asked.")
    numerical_answer: int = Field(description="The number extracted from the answer, text excluded.")
    text_answer: str = Field(description="The text part of the answer, numbers excluded.")

parser = PydanticOutputParser(pydantic_object=DesiredStructure)

In [ ]:
model = agent_foundation_model_selector.value
llm = ChatBedrock(model=model, temperature=0)

prompt_template = "Answer the user query.\n{format_instructions}\n{query}\n"
query = "Find the description of Neurons Lab services and calculate the length of this description"

**Instructions:**

1. **Set Up the Prompt:**
   - Use `PromptTemplate` to create a prompt.
   - Pass the `prompt_template` variable as the template.
   - Use `parser.get_format_instructions()` to provide the `format_instructions` as a partial variable.

2. **Combine Prompt and Model:**
   - Chain the prompt with the `llm` (language model) to create a pipeline.

3. **Invoke the Model:**
   - Use the combined prompt and model to generate output by passing `query` as input.

<details>
<summary>Click here for the solution</summary>

```python
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt_and_model = prompt | llm

output = prompt_and_model.invoke({
    "query": query
})
```

</details>

In [ ]:
print(json.loads(output.content))

Next stes: evaluation and benchmarks https://python.langchain.com/docs/langsmith/walkthrough

How to implement Q&A https://python.langchain.com/v0.2/docs/how_to/qa_chat_history_how_to/


### Deep Dive: Exploring the Inner Workings of an Agent in LangChain

This code introduces a custom **Trivia Agent** built on **LangChain** to demonstrate the intricate logic and processes involved in agent execution. The goal is to unravel the mechanics behind how an agent interprets queries, decides on actions, and uses tools to provide intelligent responses.

---

### Purpose of the Code

The code implements a step-by-step execution flow for an agent tasked with answering trivia questions. It showcases:

1. **Agent Logic and Prompting:**
   - How the agent uses a carefully structured prompt to contextualize its decisions.
   - Dynamic tool descriptions and scratchpads (intermediate state tracking) are passed to the prompt for efficient reasoning.

2. **Tool Utilization:**
   - Tools are integrated for specific tasks, such as answering trivia or providing factual data.
   - The agent selects the appropriate tool dynamically based on its reasoning.

3. **Iterative Reasoning:**
   - The agent evaluates the response at each step to determine whether it has reached a conclusion or needs to perform further actions.

4. **Transparency and Debugging:**
   - Verbose output provides insights into every stage of the agent’s execution, aiding in understanding and debugging.

---

### Key Components of the Code

#### **Prompt Template**
The prompt template defines the structure of the input passed to the language model. It includes information about tools, their names, the user’s input, and the agent's scratchpad:

#### **Core Methods**
- **Action Parsing (`get_actions`)**: Extracts actionable instructions from the language model’s response.
- **Thought Extraction (`get_last_thought`)**: Identifies and extracts the model's reasoning from its response.
- **Action Execution (`execute_action`)**: Executes the determined action using the selected tool.
- **Scratchpad Construction (`construct_scratchpad`)**: Builds a log of intermediate steps, enabling the agent to maintain a history of its reasoning and actions.

#### **Agent Execution Workflow**
The `invoke` method orchestrates the execution. It:
1. Formats the prompt with tools, user input, and the scratchpad.
2. Passes the formatted input to the language model.
3. Extracts thoughts, actions, and observations from the response.
4. Decides whether to continue or stop based on predefined criteria.
5. Updates the scratchpad with the latest state and iterates until the final answer is reached or the user halts execution.

#### **Integration with Amazon Bedrock**
The agent uses **Amazon Bedrock** for language modeling:

This integration ensures robust and scalable reasoning powered by advanced LLMs.

---

### Execution Flow

#### Input:
The user asks a trivia question, e.g., `"What is the capital of Althera?"`.

#### Step-by-Step Execution:
1. **Prompt Creation:**
   - The agent formats the prompt with the tools, scratchpad, and user input.
   
2. **Model Invocation:**
   - The language model generates a response containing a thought, an action, and an observation.

3. **Action Execution:**
   - The agent extracts the action from the response and executes it using the appropriate tool.

4. **Scratchpad Update:**
   - Intermediate results are logged in the scratchpad to provide context for the next iteration.

5. **Termination Check:**
   - The agent checks if the reasoning process has reached a conclusion. If so, it outputs the final answer.

In [ ]:
from langchain.agents import Tool
from langchain.chains import LLMChain
from langchain_core.tools.render import render_text_description
from langchain_core.messages import BaseMessage
from typing import List, Tuple, Union
from langchain.schema import AgentAction
import time
import re

react_agent_prompt = PromptTemplate(
    template=config["react_agent"]["prompt"], 
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

print(react_agent_prompt.template)

In [ ]:
class TriviaAgent:
    def __init__(self, llm, prompt_template, verbose=False):
        self.llm = llm
        self.verbose = verbose
        self.tools = self._define_tools()
        self.prompt_template = prompt_template
        self.agent = None

    def _define_tools(self):
        def trivia_knowledge_tool(input_text: str) -> str:
            if "france" in input_text.lower():
                return "Paris"
            if "althera" in input_text.lower():
                return "Althera is a fictional place, and its capital is Eldarune"
            if "oswanda" in input_text.lower():
                return "Oswanda Cape Town"
            return "I can confirm this without a doubt!"
        tools = [
            Tool(
                name="TriviaKnowledgeTool",
                func=trivia_knowledge_tool,
                description="Use this tool to retrieve factual trivia answers from a database."
            )
        ]
        return tools

    def should_continue(self, response: str):
        pattern = r"Thought:\s*I now know the final answer"
        match = re.search(pattern, response)

        if match:
            return False
        else:
            return True

    def get_actions(self, response: str, get_last: bool = True):
        pattern = r"Action:\s*(\w+)\nAction Input:\s*(.+)"
        matches = re.findall(pattern, response)

        if matches:
            action = matches[-1] if get_last else matches[0]
            return {
                "action": action[0],
                "action_input": action[1],
            }
        else:
            print("No action found in the response.")
            return False

    def get_last_thought(self, response: str, get_last: bool = True):
        pattern = r"Thought:\s*(.+?)(?=\n(?:Action|Final Answer))"
        matches = re.findall(pattern, response, re.DOTALL)

        if matches:
            thought = matches[-1] if get_last else matches[0]
            return thought.strip()
        else:
            print("No 'Thought' found in the response.")
            return False

    def execute_action(self, actions):
        if not actions or "action" not in actions or "action_input" not in actions:
            return None

        tool_name = actions["action"]
        tool_input = actions["action_input"]

        tool = next(
            (t for t in self.tools if t.name == tool_name),
            None
        )

        if not tool:
            print(f"Tool '{tool_name}' not found. Returning None.")
            return None

        tool_response = tool.run(tool_input)
        return (
            f"The determined answer is '{tool_response}'. "
            "This result is authoritative and based on verified knowledge."
        )

    def construct_scratchpad(self, intermediate_steps: List[Tuple[AgentAction, str]]) -> Union[str, List[BaseMessage]]:
        thoughts = ""
        for last_thought, action, observation in intermediate_steps:
            thoughts += f"{last_thought}\n"
            thoughts += action.log
            thoughts += f"\nObservations: {observation}\nThought:"
        return thoughts

    def invoke(self, input_str: str, max_iterations=3):
        tools = render_text_description(list(self.tools))
        tool_names = ", ".join([tool.name for tool in self.tools])

        response = ""
        agent_scratchpad = ""
        intermediate_steps = []

        print("\n--- Starting the Agent Execution ---")
        print(f"Input Question: {input_str}\n")

        step = 1

        while max_iterations <=3:
            print(f"\n--- Step {step}: Running the agent ---")

            current = self.prompt_template.format(**{
                "input": input_str,
                "tools": tools,
                "tool_names": tool_names,
                "agent_scratchpad": agent_scratchpad,
            })
            print(current)

            completions = self.llm.invoke(current)
            response = completions.content

            print(f"Agent Response:\n{response}")

            print(f"\n--- Step {step}: Extracting action ---")
            actions = self.get_actions(response)
            if actions:
                print(f"Actions: {actions}")
            else:
                print("No valid actions found.")

            print(f"\n--- Step {step}: Extracting last thought ---")
            last_thought = self.get_last_thought(response, step != 1)
            if last_thought:
                print(f"Last Thought: {last_thought}")
            else:
                print("No valid thoughts found.")

            print(f"\n--- Step {step}: Executing action ---")
            observations = self.execute_action(actions)
            if observations:
                print(f"Observations: {observations}")
            else:
                print("No valid observations found.")

            print(f"\n--- Step {step}: Verifying if the final state has been reached ---")
            if not self.should_continue(response):
                print("\n--- Final Step: The agent has reached the final answer. ---")
                break
            else:
                print("The agent has not reached the final state yet. Continuing to the next step...")

            print(f"\n--- Step {step}: Updating the scratchpad ---")
            if actions:
                phase = (
                    last_thought,
                    AgentAction(
                        tool=actions["action"],
                        tool_input=actions["action_input"],
                        log=f"""\nInvoking: `{actions["action"]}` with `{actions["action_input"]}`\n"""
                    ),
                    observations
                )

                intermediate_steps.append(phase)

            agent_scratchpad = self.construct_scratchpad(intermediate_steps)

            print("\n--- Updated Scratchpad ---")
            print(agent_scratchpad)
            print("-" * 100)

            user_input = input("\nType 'c' to continue to the next step or 'q' to quit: ").strip().lower()
            if user_input == 'q':
                print("Execution halted by the user.")
                break
            elif user_input != 'c':
                print("Invalid input. Please type 'c' to continue or 'q' to quit.")
                continue

            step += 1

        final_answer = response.split("Final Answer:")[-1].strip()
        print(f"\n--- Final Answer ---\n{final_answer}")
        return final_answer

In [ ]:
model = agent_foundation_model_selector.value
llm = ChatBedrock(temperature=0, model=model)
trivia_agent = TriviaAgent(llm, react_agent_prompt)

In [ ]:
input_question = 'What is the capital of France?'
output = trivia_agent.invoke(input_str=input_question)

In [ ]:
input_question = 'What is the capital of Oswanda?'
output = trivia_agent.invoke(input_str=input_question)

In [ ]:
input_question = 'What is the capital of Althera?'
output = trivia_agent.invoke(input_str=input_question)

### **Challenge 1: Build a Knowledgeable Agent with Context-Aware Reasoning**
**Objective:** Create an agent that performs context-aware reasoning for answering domain-specific queries.

#### **Scenario:**
You are tasked with designing an agent to help users learn about **AWS services for Machine Learning (ML)**. The agent must:
1. Maintain a short-term memory of ongoing conversations.
2. Retrieve relevant information from a pre-defined set of AWS ML-related documents.
3. Perform reasoning to generate insightful answers to user questions.

#### **Tasks:**

1. **Integrate Memory:**
   - Implement **short-term memory** to maintain conversational context.
   - Use **DuckDB** for **long-term memory** by creating a vector store of AWS ML documents. Example:
     - "AWS Machine Learning Whitepapers"
     - Documentation on **SageMaker**, **Rekognition**, or **Comprehend**.

2. **Add Tools:**
   - Create a custom tool that scrapes AWS service pages and converts the content to Markdown.
   - Integrate a **calculator tool** to perform simple computations (e.g., cost calculations).

3. **Create an Agent:**
   - Use the **ReAct Framework** to build an agent that:
     - Uses **retrieval** to fetch relevant knowledge.
     - Applies tools dynamically based on user inputs.

4. **Test the Agent:**
   - Have the agent answer these questions:
     - *"What AWS ML services can help with text analysis?"*
     - *"How can I use SageMaker for training models on large datasets?"*
     - *"What is the estimated monthly cost for using Rekognition to process 10,000 images?"*

5. **Provide the Output:**
   - Log the full execution trace, including intermediate steps, tools invoked, and final answers.

#### **Deliverable:**
- Python code implementing the memory, tools, and agent.
- Execution logs for the three sample questions.
- A brief report explaining how the agent performed reasoning and tool integration.

### **Challenge 2: Implement a Self-Improving Agent with Tree of Thoughts Framework**
**Objective:** Build an agent that uses the **Tree of Thoughts (ToT)** framework to enhance reasoning capabilities.

#### **Scenario:**
Design a self-reflective agent to recommend **best practices for ML workflows on AWS**. The agent should:
1. Generate multiple solutions for a query.
2. Evaluate and refine its responses using the **ToT Framework**.
3. Provide the most insightful and well-structured answer to the user.

#### **Tasks:**

1. **Set Up the Tree of Thoughts Framework:**
   - Implement a four-step reasoning process:
     - **Step 1:** Generate potential solutions to the query.
     - **Step 2:** Review and evaluate the solutions based on relevance and correctness.
     - **Step 3:** Deepen the reasoning for the most promising solutions.
     - **Step 4:** Rank and select the best solution.

2. **Enhance Memory:**
   - Use **short-term memory** to track intermediate reasoning steps.
   - Add **long-term memory** using DuckDB to reference prior workflows or case studies.

3. **Define a Custom Evaluation Metric:**
   - Create a scoring system for evaluating solutions:
     - Completeness: Does the solution address all parts of the query?
     - Relevance: How closely does it align with AWS ML best practices?
     - Clarity: Is the solution easy to understand?

4. **Test the Agent:**
   - Provide recommendations for the following queries:
     - *"What are the best practices for feature engineering using SageMaker?"*
     - *"How can I implement a distributed training pipeline on AWS?"*

5. **Provide the Output:**
   - Log the agent's intermediate thoughts and the final ranked solutions.
   - Include the scores for each solution based on the evaluation metric.

#### **Deliverable:**
- Python code implementing the Tree of Thoughts framework.
- Execution logs showing intermediate steps, evaluations, and final recommendations.
- A brief report discussing how the agent used reasoning and memory to improve its output.

### Conclusion

This notebook serves as a comprehensive guide to building robust LLM-powered applications with Amazon Bedrock. By the end, you'll understand how to design agents that are not only knowledgeable but also capable of reasoning, remembering, and interacting with external systems effectively.

### Key Concepts Covered:
- **Agents:** Autonomous systems that perform reasoning and interact with tools to solve user queries.
- **Memory:** Techniques for storing and retrieving data to improve LLM contextuality.
- **Tools:** External functions (e.g., search engines, APIs) that agents can leverage.
- **ReAct Framework:** A methodology where agents reason and act iteratively based on intermediate outputs.

### Structure of the Notebook:
1. **Setup:** Load configurations, dependencies, and initialize tools with Amazon Bedrock.
2. **Tree of Thoughts Framework:** Implement CoT techniques for multi-step reasoning.
3. **Memory:** Add sensory, short-term, and long-term memory to agents.
4. **Tool Integration:** Use existing and custom tools to enhance LLM capabilities.
5. **Agent Construction:** Combine tools, memory, and reasoning to create intelligent agents using Amazon Bedrock.
6. **Evaluation:** Test agents with complex queries and structured outputs.